# Database Table Loader
This notebook connects to the local PostgreSQL database, lists all tables, and loads a selected table into a pandas DataFrame.

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine, inspect

# Load environment variables
load_dotenv(override=True)

DB_HOST = os.getenv('LOCAL_HOST')
DB_NAME = os.getenv('LOCAL_DB')
DB_USER = os.getenv('LOCAL_USER')
DB_PW = os.getenv('LOCAL_PW')
DB_PORT = os.getenv('LOCA_PORT', '5432')

engine = create_engine(f'postgresql://{DB_USER}:{DB_PW}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

In [2]:
# List all tables in the database
inspector = inspect(engine)
tables = inspector.get_table_names()
print('Tables in database:', tables)

Tables in database: ['urban_data_expanded', 'urban_data_directory', 'urban_institute_data', 'census_data', 'location_data']


In [3]:
# Load all tables into pandas DataFrames
dfs = {}
for table in tables:
    df = pd.read_sql_table(table, engine)
    dfs[table] = df
    print(f"Loaded table: {table}, shape: {df.shape}")
if not tables:
    print('No tables found in the database.')

Loaded table: urban_data_expanded, shape: (220000, 33)
Loaded table: urban_data_directory, shape: (90000, 57)
Loaded table: census_data, shape: (333160, 16)
Loaded table: location_data, shape: (27796, 7)
Loaded table: urban_institute_data, shape: (310000, 8)


In [4]:
census_data = dfs['census_data']

school_directory = dfs['urban_data_directory']
school_directory = school_directory.query('`latitude`!="" and `longitude`!=""')
school_directory[['latitude', 'longitude']] = school_directory[['latitude', 'longitude']].astype(float)
school_data = dfs['urban_data_expanded']
location_data = dfs['location_data']


C:\Users\Leo\AppData\Local\Temp\ipykernel_25356\2160154767.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_directory[['latitude', 'longitude']] = school_directory[['latitude', 'longitude']].astype(float)


In [5]:
school_location = pd.merge(school_directory, 
         location_data, 
         on=['latitude', 'longitude'], 
         how='left'
)
school_facts = school_location[['city_mailing', 'county', 'state', 'zip_mailing', 
                 'leaid', "ncessch",'school_name','school_status','school_type',
                 'enrollment','year']]

In [6]:
school_assessments = school_data[['lea_name','leaid','ncessch','year', 
             'econ_disadvantaged', 'enrollment','grade',
             'math_test_num_valid', 'math_test_pct_prof_high', 'math_test_pct_prof_low', 'math_test_pct_prof_midpt',
             'migrant','race', 'read_test_num_valid', 'read_test_pct_prof_high', 'read_test_pct_prof_low', 'read_test_pct_prof_midpt',
             'sex']]

In [9]:
data= pd.merge(school_facts, 
         school_assessments, 
         on=['leaid','ncessch','year'], 
         how='right'
)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220000 entries, 0 to 219999
Data columns (total 26 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   city_mailing              33862 non-null   object
 1   county                    33862 non-null   object
 2   state                     33862 non-null   object
 3   zip_mailing               33862 non-null   object
 4   leaid                     220000 non-null  object
 5   ncessch                   220000 non-null  object
 6   school_name               33862 non-null   object
 7   school_status             33862 non-null   object
 8   school_type               33862 non-null   object
 9   enrollment_x              33862 non-null   object
 10  year                      220000 non-null  int64 
 11  lea_name                  220000 non-null  object
 12  econ_disadvantaged        220000 non-null  object
 13  enrollment_y              220000 non-null  object
 14  grad